# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'05-01-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'05-01-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-05-02 04:20:48,33.93911,67.709953,59939,2631,53272.0,4036.0,Afghanistan,153.972655,4.389463
1,NaN,NaN,NaN,Albania,2021-05-02 04:20:48,41.15330,20.168300,131185,2396,110172.0,18617.0,Albania,4558.516923,1.826428
2,NaN,NaN,NaN,Algeria,2021-05-02 04:20:48,28.03390,1.659600,122311,3261,85249.0,33801.0,Algeria,278.923810,2.666154
3,NaN,NaN,NaN,Andorra,2021-05-02 04:20:48,42.50630,1.521800,13232,125,12684.0,423.0,Andorra,17125.477254,0.944680
4,NaN,NaN,NaN,Angola,2021-05-02 04:20:48,-11.20270,17.873900,26815,600,23913.0,2302.0,Angola,81.588211,2.237554


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,58312,58542,58730,58843,59015,59225,59370,59576,59745,59939
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,130114,130270,130409,130537,130606,130736,130859,130977,131085,131185
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,120363,120562,120736,120922,121112,121344,121580,121866,122108,122311


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2561,2565,2572,2582,2592,2598,2611,2618,2625,2631
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2364,2367,2372,2378,2379,2383,2386,2389,2394,2396
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3181,3190,3198,3207,3217,3225,3234,3244,3253,3261


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,52348,52363,52392,52489,52520,52794,52974,53103,53206,53272
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,103582,104278,105016,105728,106465,107163,107887,108606,109338,110172
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,83900,84038,84167,84299,84440,84598,84745,84940,85108,85249


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6819,6835,6876,6879,6882,6889,6890,6897,6904,6907
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20815,20833,20838,20847,20863,20875,20897,20921,20941,20966
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2289,2292,2296,2296,2296,2297,2298,2299,2300,2302


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,107,107,107,107,107,107,107,107,107,107
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,303,303,305,305,305,305,305,305,305,306
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,56,56,56,56,56,56,56,56,56,56


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-05-02 04:20:48,32.539527,-86.644082,6907,107,NaN,NaN,"Autauga, Alabama, US",12362.848807,1.549153
689,1075.0,Lamar,Alabama,US,2021-05-02 04:20:48,33.779950,-88.096680,1409,34,NaN,NaN,"Lamar, Alabama, US",10206.446940,2.413059
690,1077.0,Lauderdale,Alabama,US,2021-05-02 04:20:48,34.901719,-87.656247,9386,238,NaN,NaN,"Lauderdale, Alabama, US",10121.968316,2.535691


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,32392001,576722,0.0
India,19164969,211853,15684406.0
Brazil,14725975,406437,13023892.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,32392001,576722,0.0,31815279.0,2021-05-02 04:20:48,37.950547,-91.419547
India,19164969,211853,15684406.0,3268710.0,2021-05-02 04:20:48,23.088275,81.806127
Brazil,14725975,406437,13023892.0,1295646.0,2021-05-02 04:20:48,-12.669522,-48.480493
France,5703505,104867,347827.0,5250811.0,2021-05-02 04:20:48,6.430808,-34.730285
Turkey,4849408,40504,4405199.0,403705.0,2021-05-02 04:20:48,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3743408,61866,0.0
Texas,2895810,50274,0.0
Florida,2238937,35239,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3743408,61866,0.0,3681542.0,2021-05-02 04:20:48,37.843962,-120.728594
Texas,2895810,50274,0.0,2845536.0,2021-05-02 04:20:48,31.660643,-98.653069
Florida,2238937,35239,0.0,2203698.0,2021-05-02 04:20:48,28.940755,-82.700744
New York,2052022,52309,0.0,1999713.0,2021-05-02 04:20:48,42.544151,-75.474183
Illinois,1337760,24330,0.0,1313430.0,2021-05-02 04:20:48,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1233488,23919,0.0
Arizona,Maricopa,537899,9876,0.0
Illinois,Cook,535614,9960,0.0
Florida,Miami-Dade,484514,6175,0.0
Texas,Harris,392664,6212,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1233488,23919,0.0,1209569.0,2021-05-02 04:20:48,34.308284,-118.228241,6037.0
Arizona,Maricopa,537899,9876,0.0,528023.0,2021-05-02 04:20:48,33.348359,-112.491815,4013.0
Illinois,Cook,535614,9960,0.0,525654.0,2021-05-02 04:20:48,41.841448,-87.816588,17031.0
Florida,Miami-Dade,484514,6175,0.0,478339.0,2021-05-02 04:20:48,25.611236,-80.551706,12086.0
Texas,Harris,392664,6212,0.0,386452.0,2021-05-02 04:20:48,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-29,59576,130977,121866,13198,26431,1232,2954943,215528,29798,616739,...,4429849,195734,90406,4,196386,2910,295601,6294,91533,38235
2021-04-30,59745,131085,122108,13232,26652,1232,2977363,216064,29811,618870,...,4432246,198428,90808,4,197683,2928,296462,6317,91586,38257
2021-05-01,59939,131185,122311,13232,26815,1232,2993865,216596,29824,620485,...,4434157,200908,91250,4,198865,2942,296462,6325,91670,38260


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-29,2618,2389,3244,125,594,32,63508,4087,910,10179,...,127759,2563,647,1,2117,35,3231,1222,1250,1567
2021-04-30,2625,2394,3253,125,596,32,63865,4108,910,10205,...,127775,2616,649,1,2136,35,3249,1226,1251,1567
2021-05-01,2631,2396,3261,125,600,32,64096,4128,910,10233,...,127782,2669,650,1,2155,35,3249,1229,1251,1568


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-29,53103,108606,84940,12650,23606,1014,2608077,197171,23279,582575,...,14658,164965,86774,3,178072,2516,271233,2731,89852,35594
2021-04-30,53206,109338,85108,12684,23876,1014,2634306,198135,23294,584777,...,14683,167823,87067,3,179200,2516,272333,2751,89933,35612
2021-05-01,53272,110172,85249,12684,23913,1014,2655359,199115,23308,587115,...,14684,170750,87313,3,180301,2548,272333,2794,90012,35620


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6819,6835,6876,6879,6882,6889,6890,6897,6904,6907
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1398,1399,1403,1404,1404,1404,1406,1406,1408,1409
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9301,9310,9317,9322,9337,9350,9358,9367,9376,9386
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2977,2982,2982,2983,2984,2988,2988,2990,2992,2994
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15694,15709,15722,15723,15731,15742,15761,15773,15784,15792


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-29,527513,68002,0,861653,335529,3740038,509194,338447,104038,49,...,846472,2890257,396985,22824,3093,658341,401718,152733,659600,58069
2021-04-30,527922,68148,0,862497,335725,3742115,510966,339233,104405,49,...,847430,2893928,397323,22952,3125,659590,403040,153128,660450,58142
2021-05-01,528309,68148,0,863571,335968,3743408,512804,339233,104621,49,...,847430,2895810,397733,23032,3125,660553,404709,153498,661209,58142


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-04-29        6897   20921    2299  2593   6607    1227   2151   14432   
2021-04-30        6904   20941    2300  2594   6613    1228   2152   14438   
2021-05-01        6907   20966    2302  2596   6616    1230   2154   14447   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-29         3540     1835  ...    2700    623     3122      776   
2021-04-30         3541     1836  ...    2701    623     3124      776   
2021-05-01         3544     1837  ...    2701    623     3124      776   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-29           4219  3730  2181          0      903    650  
2021-04-30           4230  3732  2183          0      903    650  
2021-05-01           4230  3732  2183          0      903    650  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-29,10887,347,0,17305,5735,61672,6284,8084,1624,0,...,12188,50164,2197,246,27,10751,5487,2674,7542,707
2021-04-30,10896,347,0,17324,5739,62078,6295,8097,1625,0,...,12197,50219,2202,246,27,10770,5499,2679,7550,707
2021-05-01,10913,347,0,17338,5741,61866,6302,8097,1626,0,...,12197,50274,2203,247,27,10777,5499,2679,7569,707


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-04-29         107     305      56   63    135      40     69     312   
2021-04-30         107     305      56   63    135      40     69     312   
2021-05-01         107     306      56   63    135      41     69     312   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-29          121       45  ...      30     11       31        7   
2021-04-30          122       45  ...      30     11       31        7   
2021-05-01          123       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-29             37     9    12          0       26      5  
2021-04-30             37     9    12          0       26      5  
2021-05-01             37     9    12          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-29,0.003470,0.000902,0.002352,0.003803,0.010050,0.0,0.008895,0.003053,0.000840,0.003627,...,0.000555,0.014024,0.004422,0.0,0.007319,0.015707,0.003568,0.004950,0.000536,0.001152
2021-04-30,0.002837,0.000825,0.001986,0.002576,0.008361,0.0,0.007587,0.002487,0.000436,0.003455,...,0.000541,0.013764,0.004447,0.0,0.006604,0.006186,0.002913,0.003654,0.000579,0.000575
2021-05-01,0.003247,0.000763,0.001662,0.000000,0.006116,0.0,0.005542,0.002462,0.000436,0.002610,...,0.000431,0.012498,0.004867,0.0,0.005979,0.004781,0.000000,0.001266,0.000917,0.000078


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-29,0.002681,0.001257,0.003092,0.0,0.005076,0.0,0.008912,0.003930,0.0,0.002660,...,0.000196,0.026432,0.001548,0.0,0.008576,0.0,0.007798,0.004934,0.000801,0.001278
2021-04-30,0.002674,0.002093,0.002774,0.0,0.003367,0.0,0.005621,0.005138,0.0,0.002554,...,0.000125,0.020679,0.003091,0.0,0.008975,0.0,0.005571,0.003273,0.000800,0.000000
2021-05-01,0.002286,0.000835,0.002459,0.0,0.006711,0.0,0.003617,0.004869,0.0,0.002744,...,0.000055,0.020260,0.001541,0.0,0.008895,0.0,0.000000,0.002447,0.000000,0.000638


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-29,0.002435,0.006664,0.002301,0.004766,0.000933,0.0,0.010035,0.004493,0.000344,0.003868,...,0.002599,0.015751,0.003759,0.0,0.005591,0.000000,0.011860,0.011481,0.001215,0.002168
2021-04-30,0.001940,0.006740,0.001978,0.002688,0.011438,0.0,0.010057,0.004889,0.000644,0.003780,...,0.001706,0.017325,0.003377,0.0,0.006335,0.000000,0.004056,0.007323,0.000901,0.000506
2021-05-01,0.001240,0.007628,0.001657,0.000000,0.001550,0.0,0.007992,0.004946,0.000601,0.003998,...,0.000068,0.017441,0.002825,0.0,0.006144,0.012719,0.000000,0.015631,0.000878,0.000225


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-29,0.000816,0.002477,NaN,0.001023,0.000716,0.000458,0.005507,0.001438,0.001868,0.0,...,0.001292,0.001254,0.001168,0.004445,0.000000,0.001806,0.003921,0.002836,0.001372,0.001207
2021-04-30,0.000775,0.002147,NaN,0.000980,0.000584,0.000555,0.003480,0.002322,0.003528,0.0,...,0.001132,0.001270,0.000851,0.005608,0.010346,0.001897,0.003291,0.002586,0.001289,0.001257
2021-05-01,0.000733,0.000000,NaN,0.001245,0.000724,0.000346,0.003597,0.000000,0.002069,0.0,...,0.000000,0.000650,0.001032,0.003486,0.000000,0.001460,0.004141,0.002416,0.001149,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-29      0.001016  0.001148  0.000435  0.000772  0.001820  0.000000   
2021-04-30      0.001015  0.000956  0.000435  0.000386  0.000908  0.000815   
2021-05-01      0.000435  0.001194  0.000870  0.000771  0.000454  0.001629   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-04-29      0.001397  0.000763  0.000848  0.000545  ...  0.001484   
2021-04-30      0.000465  0.000416  0.000282  0.000545  ...  0.000370   
2021-05-01      0.000929  0.000623  0.000847  0.000545  ...  0.000000   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2021-04-29      0.001608  0.000962  0.00129   0.003329  0.000268  0.000459   
2021-04-30      0.000000  0.000641  0.00000   0.002607  0.000536  0.000917   
2021-05-01      0.000000  0.000000  0.00000   0.000000  0.000000  0.000000   

Province_State                                
Admin2         Unassigned Washakie    Weston  
2021-04-29            NaN      0.0  0.001541  
2021-04-30            NaN      0.0  0.000000  
2021-05-01            NaN      0.0  0.000000  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-29,0.000735,0.0,NaN,0.001331,0.001572,0.001380,0.001913,0.000495,0.001233,NaN,...,0.001397,0.001317,0.003196,0.000000,0.0,0.001490,0.002375,0.000374,0.001062,0.0
2021-04-30,0.000827,0.0,NaN,0.001098,0.000697,0.006583,0.001750,0.001608,0.000616,NaN,...,0.000738,0.001096,0.002276,0.000000,0.0,0.001767,0.002187,0.001870,0.001061,0.0
2021-05-01,0.001560,0.0,NaN,0.000808,0.000348,-0.003415,0.001112,0.000000,0.000615,NaN,...,0.000000,0.001095,0.000454,0.004065,0.0,0.000650,0.000000,0.000000,0.002517,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2021-04-29         0.0  0.000000     0.0  0.0  0.007463   0.000    0.0   
2021-04-30         0.0  0.000000     0.0  0.0  0.000000   0.000    0.0   
2021-05-01         0.0  0.003279     0.0  0.0  0.000000   0.025    0.0   

Province_State                             ... Wyoming                  \
Admin2         Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2021-04-29         0.0  0.000000      0.0  ...     0.0    0.0      0.0   
2021-04-30         0.0  0.008264      0.0  ...     0.0    0.0      0.0   
2021-05-01         0.0  0.008197      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-29          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-30          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-05-01          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-29,0.003132,0.000908,0.002096,0.003056,0.008979,0.000555,0.008441,0.003046,0.000865,0.003483,...,0.000534,0.014520,0.004135,0.000651,0.006735,0.009145,0.003769,0.005828,0.000554,0.001026
2021-04-30,0.002984,0.000866,0.002041,0.002816,0.008670,0.000278,0.008014,0.002766,0.000651,0.003469,...,0.000537,0.014142,0.004291,0.000326,0.006670,0.007665,0.003341,0.004741,0.000566,0.000801
2021-05-01,0.003116,0.000815,0.001852,0.001408,0.007393,0.000139,0.006778,0.002614,0.000543,0.003039,...,0.000484,0.013320,0.004579,0.000163,0.006324,0.006223,0.001670,0.003004,0.000742,0.000440


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-29,0.003314,0.001320,0.002917,0.002144,0.005785,0.002016,0.007613,0.003942,4.196587e-09,0.002510,...,0.000185,0.024225,0.001253,0.0,0.008519,4.501836e-74,0.005628,0.006055,0.000668,0.001187
2021-04-30,0.002994,0.001706,0.002846,0.001072,0.004576,0.001008,0.006617,0.004540,2.098293e-09,0.002532,...,0.000155,0.022452,0.002172,0.0,0.008747,2.250918e-74,0.005600,0.004664,0.000734,0.000594
2021-05-01,0.002640,0.001271,0.002652,0.000536,0.005644,0.000504,0.005117,0.004704,1.049147e-09,0.002638,...,0.000105,0.021356,0.001856,0.0,0.008821,1.125459e-74,0.002800,0.003556,0.000367,0.000616


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-29,0.002834,0.006689,0.002021,0.003706,0.003978,0.001616,0.009062,0.004059,0.000472,0.004052,...,0.003110,0.015562,0.003294,0.015625,0.005810,0.000334,0.008828,0.013669,0.000915,0.002598
2021-04-30,0.002387,0.006715,0.002000,0.003197,0.007708,0.000808,0.009560,0.004474,0.000558,0.003916,...,0.002408,0.016444,0.003335,0.007812,0.006072,0.000167,0.006442,0.010496,0.000908,0.001552
2021-05-01,0.001814,0.007171,0.001828,0.001598,0.004629,0.000404,0.008776,0.004710,0.000580,0.003957,...,0.001238,0.016942,0.003080,0.003906,0.006108,0.006443,0.003221,0.013064,0.000893,0.000888


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-29,0.000740,0.002646,NaN,0.000892,0.000697,0.000476,0.004161,0.001779,0.002020,2.574862e-124,...,0.001335,0.001306,0.001110,0.003428,0.000607,0.001710,0.003800,0.002574,0.001275,0.001399
2021-04-30,0.000758,0.002397,NaN,0.000936,0.000641,0.000516,0.003820,0.002051,0.002774,1.287431e-124,...,0.001233,0.001288,0.000981,0.004518,0.005476,0.001804,0.003545,0.002580,0.001282,0.001328
2021-05-01,0.000745,0.001198,NaN,0.001091,0.000682,0.000431,0.003709,0.001025,0.002421,6.437154e-125,...,0.000617,0.000969,0.001006,0.004002,0.002738,0.001632,0.003843,0.002498,0.001216,0.000664


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-29      0.000847  0.000991  0.000437  0.000941  0.001626  0.000550   
2021-04-30      0.000931  0.000974  0.000436  0.000663  0.001267  0.000683   
2021-05-01      0.000683  0.001084  0.000653  0.000717  0.000860  0.001156   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-04-29      0.000894  0.000630  0.000654  0.000426  ...  0.002902   
2021-04-30      0.000680  0.000523  0.000468  0.000486  ...  0.001636   
2021-05-01      0.000804  0.000573  0.000658  0.000515  ...  0.000818   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-04-29      0.001523  0.000807  0.002196   0.003368  0.000537  0.001709   
2021-04-30      0.000762  0.000724  0.001098   0.002988  0.000537  0.001313   
2021-05-01      0.000381  0.000362  0.000549   0.001494  0.000268  0.000656   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-04-29           -1.0  0.000973  0.001169  
2021-04-30           -1.0  0.000486  0.000585  
2021-05-01           -1.0  0.000243  0.000292  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-29,0.001009,0.002435,NaN,0.000860,0.000980,0.001064,0.001568,0.000831,0.001534,NaN,...,0.001046,0.001245,0.002304,0.001606,0.000019,0.001404,0.002218,-0.005724,0.002279,0.000357
2021-04-30,0.000918,0.001217,NaN,0.000979,0.000839,0.003824,0.001659,0.001219,0.001075,NaN,...,0.000892,0.001171,0.002290,0.000803,0.000009,0.001586,0.002203,-0.001927,0.001670,0.000179
2021-05-01,0.001239,0.000609,NaN,0.000894,0.000594,0.000204,0.001386,0.000610,0.000845,NaN,...,0.000446,0.001133,0.001372,0.002434,0.000005,0.001118,0.001101,-0.000963,0.002093,0.000089


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2021-04-29      0.000022  0.000116  0.000071  0.004101  0.003789   
2021-04-30      0.000011  0.000058  0.000036  0.002050  0.001894   
2021-05-01      0.000005  0.001668  0.000018  0.001025  0.000947   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2021-04-29      2.388009e-11  0.000232 -0.000037 -0.000032  0.000022  ...   
2021-04-30      1.194005e-11  0.000116 -0.000019  0.004116  0.000011  ...   
2021-05-01      1.250000e-02  0.000058 -0.000009  0.006156  0.000006  ...   

Province_State       Wyoming                                            \
Admin2                  Park        Platte      Sheridan      Sublette   
2021-04-29      1.254641e-13  5.092538e-30  9.475831e-16  4.162818e-36   
2021-04-30      6.273203e-14  2.546269e-30  4.737915e-16  2.081409e-36   
2021-05-01      3.136601e-14  1.273135e-30  2.368958e-16  1.040704e-36   

Province_State                                                       \
Admin2            Sweetwater         Teton         Uinta Unassigned   
2021-04-29      2.587007e-11  1.328850e-23  6.753133e-31  -0.501961   
2021-04-30      1.293504e-11  6.644248e-24  3.376566e-31  -0.501961   
2021-05-01      6.467518e-12  3.322124e-24  1.688283e-31  -0.501961   

Province_State                              
Admin2              Washakie        Weston  
2021-04-29      2.586429e-28  5.492996e-27  
2021-04-30      1.293214e-28  2.746498e-27  
2021-05-01      6.466072e-29  1.373249e-27  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
